In [3]:
!pip install opencv-python

In [5]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os

In [7]:
mydir = r"H:\Git\DL-Data-Sets\dogs_vs_cats\train"

In [9]:
categories = ['cats','dogs']

In [13]:
data =[]

for i in categories:
    folder_path = os.path.join(mydir,i)
    
    if i == 'cats':
        label = 0
    else:
        label = 1

    for j in os.listdir(folder_path):

        img_path = os.path.join(folder_path,j)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(150,150))
        data.append([img,label])

In [17]:
import random
random.shuffle(data)

In [19]:
x = []
y = []

for i in data:
    x.append(i[0])
    y.append(i[1])

In [23]:
import numpy as np
x = np.array(x)
y = np.array(y)

In [25]:
x.shape

(20000, 150, 150, 3)

In [27]:
x = x/255

In [33]:
# create CNN
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

C:\Users\User\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [39]:
model.fit(x,y,epochs=5,validation_split=0.1)

Epoch 1/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 106s 180ms/step - accuracy: 0.5054 - loss: 0.6930 - val_accuracy: 0.5610 - val_loss: 0.6824
Epoch 2/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 100s 178ms/step - accuracy: 0.6060 - loss: 0.6674 - val_accuracy: 0.6760 - val_loss: 0.6208
Epoch 3/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 99s 175ms/step - accuracy: 0.6789 - loss: 0.6221 - val_accuracy: 0.6910 - val_loss: 0.6268
Epoch 4/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 100s 178ms/step - accuracy: 0.7315 - loss: 0.5730 - val_accuracy: 0.7145 - val_loss: 0.5837
Epoch 5/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 110s 195ms/step - accuracy: 0.7697 - loss: 0.5204 - val_accuracy: 0.7735 - val_loss: 0.5070


In [45]:
# use data Augmentation for better accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [51]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r"H:\Git\DL-Data-Sets\dogs_vs_cats\train",  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"H:\Git\DL-Data-Sets\dogs_vs_cats\test",
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [59]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [61]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [67]:
model.fit(
    train_generator,
    steps_per_epoch=2000,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=800
)
#model.save_weights('first_try.h5')  # always save your weights after training or during training

1250/2000 ━━━━━━━━━━━━━━━━━━━━ 2:08 172ms/step - accuracy: 0.7314 - loss: 0.5485

C:\Users\User\anaconda3\envs\tf_env\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 233s 116ms/step - accuracy: 0.7371 - loss: 0.5410 - val_accuracy: 0.7942 - val_loss: 0.4538
